# Imports
Ici on peut retrouver les importations nécéssaires pour le projet

In [42]:
import nltk
nltk.download()
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


# Porter
Ici nous avons le stemmer Porter et nous faisons quelques tests pour vérifier qu'il fonctionne

In [36]:
porter = PorterStemmer()
print("Porter Stemmer")
print(porter.stem("cats"))
print(porter.stem("trouble"))
print(porter.stem("troubling"))
print(porter.stem("troubled"))

Porter Stemmer
cat
troubl
troubl
troubl


In [37]:
# https://scikit-learn.org/stable/modules/feature_extraction.html
corpus = []
f = open("data/interest.acl94.txt", "r")
for x in f:
    corpus.append(x)

# We remove all the $$ from the list to make it cleaner
corpus = [i for i in corpus if i != "$$\n"]
print(corpus[1])

[ longer/JJR maturities/NNS ] are/VBP thought/VBN to/TO indicate/VB [ declining/VBG interest_6/NN rates/NNS ] because/IN [ they/PP ] permit/VBP [ portfolio/NN managers/NNS ] to/TO retain/VB relatively/RB [ higher/JJR rates/NNS ] for/IN [ a/DT longer/JJR period/NN ] ./. 



# Count Vectorizer

In [38]:
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
vectorizer = CountVectorizer(input="content")
X_count = vectorizer.fit_transform(corpus)
vectorizer.get_feature_names_out()
print(X_count.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


# TF-IDF Vectorizer

In [39]:
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer
Tfvectorizer = TfidfVectorizer(input="content")
X_tfidf = vectorizer.fit_transform(corpus)
vectorizer.get_feature_names_out()
print(X_tfidf.shape)
print(X_tfidf.toarray())

(2368, 7073)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [43]:
# https://www.guru99.com/pos-tagging-chunking-nltk.html
# https://www.datacamp.com/tutorial/stemming-lemmatization-python
#text = "learn php from guru99"
tokens = nltk.word_tokenize(corpus[1])
print(tokens)
tag = nltk.pos_tag(tokens)
print(tag)
grammar = "NP: {<DT>?<JJ>*<NN>}"
cp  =nltk.RegexpParser(grammar)
result = cp.parse(tag)
print(result)
result.draw()    # It will draw the pattern graphically which can be seen in Noun Phrase chunking

['[', 'longer/JJR', 'maturities/NNS', ']', 'are/VBP', 'thought/VBN', 'to/TO', 'indicate/VB', '[', 'declining/VBG', 'interest_6/NN', 'rates/NNS', ']', 'because/IN', '[', 'they/PP', ']', 'permit/VBP', '[', 'portfolio/NN', 'managers/NNS', ']', 'to/TO', 'retain/VB', 'relatively/RB', '[', 'higher/JJR', 'rates/NNS', ']', 'for/IN', '[', 'a/DT', 'longer/JJR', 'period/NN', ']', './', '.']
[('[', 'JJ'), ('longer/JJR', 'NN'), ('maturities/NNS', 'NN'), (']', 'NNP'), ('are/VBP', 'NN'), ('thought/VBN', 'NN'), ('to/TO', 'NN'), ('indicate/VB', 'NN'), ('[', 'NNP'), ('declining/VBG', 'NN'), ('interest_6/NN', 'NN'), ('rates/NNS', 'NN'), (']', 'NNP'), ('because/IN', 'NN'), ('[', 'NNP'), ('they/PP', 'NN'), (']', 'NNP'), ('permit/VBP', 'NN'), ('[', 'NNP'), ('portfolio/NN', 'NN'), ('managers/NNS', 'NN'), (']', 'NNP'), ('to/TO', 'NN'), ('retain/VB', 'NN'), ('relatively/RB', 'NN'), ('[', 'NNP'), ('higher/JJR', 'NN'), ('rates/NNS', 'NN'), (']', 'NNP'), ('for/IN', 'NN'), ('[', 'NNP'), ('a/DT', 'NN'), ('longer/JJ